In [16]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm

insurance_df = pd.read_csv("../Data/insurance.csv")

In [17]:
insurance_df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


### Data Splitting

In [21]:
from sklearn.model_selection import train_test_split

insurance_df["smoker_flag"] = np.where(insurance_df["smoker"] == "yes", 1, 0)

features = ["age", "bmi", "children", "smoker_flag"]

X = sm.add_constant(insurance_df[features])
y = insurance_df["charges"]

X, X_test, y, y_test = train_test_split(X, y, test_size=.2, random_state=2023)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=.25, random_state=2024)

In [22]:
model = sm.OLS(y, X).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                charges   R-squared:                       0.748
Model:                            OLS   Adj. R-squared:                  0.747
Method:                 Least Squares   F-statistic:                     788.6
Date:                Wed, 05 Nov 2025   Prob (F-statistic):          1.66e-316
Time:                        12:00:24   Log-Likelihood:                -10855.
No. Observations:                1070   AIC:                         2.172e+04
Df Residuals:                    1065   BIC:                         2.174e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -1.217e+04   1067.908    -11.400      0.000   -1.43e+04   -1.01e+04
age           262.4888     13.506     19.436      0.000     235.988     288.989
bmi           318.3438     30.887     10.307      0.000     257.738     378.949
children      454.4452    158.307      2.871      0.004     143.817     765.074
smoker_flag  2.396e+04    469.377     51.047      0.000     2.3e+04    2.49e+04
==============================================================================
Omnibus:                      254.675   Durbin-Watson:                   1.941
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              632.310
Skew:                           1.255   Prob(JB):                    4.96e-138
Kurtosis:                       5.807   Cond. No.                         291.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [23]:
from sklearn.metrics import r2_score as r2
from sklearn.metrics import mean_absolute_error as mae
print(r2(y_test, model.predict(X_test)))
print(mae(y_test, model.predict(X_test)))


0.7589480825048381
4042.196079549451


### Validation Scoring

In [10]:
model = sm.OLS(y_train, X_train).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                charges   R-squared:                       0.762
Model:                            OLS   Adj. R-squared:                  0.760
Method:                 Least Squares   F-statistic:                     636.2
Date:                Wed, 05 Nov 2025   Prob (F-statistic):          2.52e-246
Time:                        11:50:04   Log-Likelihood:                -8114.9
No. Observations:                 802   AIC:                         1.624e+04
Df Residuals:                     797   BIC:                         1.626e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -1.254e+04   1249.630    -10.037      0.000    -1.5e+04   -1.01e+04
age           271.3667     15.500     17.507      0.000     240.941     301.792
bmi           312.6168     35.190      8.884      0.000     243.541     381.693
children      440.1047    175.402      2.509      0.012      95.801     784.409
smoker_flag  2.443e+04    526.964     46.366      0.000    2.34e+04    2.55e+04
==============================================================================
Omnibus:                      179.479   Durbin-Watson:                   1.957
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              449.874
Skew:                           1.160   Prob(JB):                     2.05e-98
Kurtosis:                       5.843   Cond. No.                         304.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [24]:
from sklearn.metrics import r2_score as r2
from sklearn.metrics import mean_absolute_error as mae

print(f"Train R2: {r2(y_train, model.predict(X_train))}")
print(f"Train MAE: {mae(y_train, model.predict(X_train))}")
print(f"Validation R2: {r2(y_valid, model.predict(X_valid))}")
print(f"Validation MAE: {mae(y_valid, model.predict(X_valid))}")


Train R2: 0.7457363087276341
Train MAE: 4229.8102947411635
Validation R2: 0.7529567406177518
Validation MAE: 4105.95220058447


### Final Fit (on all training data) & Test Score

In [9]:
model = sm.OLS(y, X).fit()

print(f"Train R2: {r2(y, model.predict(X))}")
print(f"Train MAE: {mae(y, model.predict(X))}")

Train R2: 0.7475937041957614
Train MAE: 4198.787893587897


In [10]:
print(f"Test R2: {r2(y_test, model.predict(X_test))}")
print(f"Test MAE: {mae(y_test, model.predict(X_test))}")

Test R2: 0.7589480825048381
Test MAE: 4042.196079549451


### Cross Validation Loop

In [8]:
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score as r2


kf = KFold(n_splits=5, shuffle=True, random_state=2023)

# Create a list to store validation scores for each fold
cv_lm_r2s = []
cv_lm_mae = []

# Loop through each fold in X and y
for train_ind, val_ind in kf.split(X, y):
    # Subset data based on CV folds
    X_train, y_train = X.iloc[train_ind], y.iloc[train_ind]
    X_val, y_val = X.iloc[val_ind], y.iloc[val_ind]
    # Fit the Model on fold's training data
    model = sm.OLS(y_train, X_train).fit()
    # Append Validation score to list 
    cv_lm_r2s.append(r2(y_val, model.predict(X_val),))
    cv_lm_mae.append(mae(y_val, model.predict(X_val),))

print("All Validation R2s: ", [round(x, 3) for x in cv_lm_r2s])
print(f"Cross Val R2s: {round(np.mean(cv_lm_r2s), 3)} +- {round(np.std(cv_lm_r2s), 3)}")

print("All Validation MAEs: ", [round(x, 3) for x in cv_lm_mae])
print(f"Cross Val MAEs: {round(np.mean(cv_lm_mae), 3)} +- {round(np.std(cv_lm_mae), 3)}")

All Validation R2s:  [0.736, 0.752, 0.756, 0.77, 0.714]
Cross Val R2s: 0.745 +- 0.019
All Validation MAEs:  [4363.249, 3995.482, 4385.796, 3958.32, 4418.007]
Cross Val MAEs: 4224.171 +- 202.984


### Final Fit & Score On Test

In [9]:
model = sm.OLS(y, X).fit()

r2(y_test, model.predict(X_test))

0.7589480825048381